In [19]:
# Import packages
import pandas as pd
import numpy as np

In [20]:
# Load data for foodcodes that were replaced with FNDDS 2018
new_codes = pd.read_csv('fndds_missing_foods_replaced.csv')
# Load ASA24 dataset
asa24 = pd.read_csv('fl100_recalls_qcd.csv')

In [21]:
new_codes['NewFoodCode'].sort_values(ascending=True)

332    11350000
76     11360000
108    11411100
482    11411200
83     11411200
         ...   
117    95201200
330    95230010
292    99995000
151    99997410
86     99997550
Name: NewFoodCode, Length: 483, dtype: int64

In [22]:
new_codes = new_codes[new_codes['NewFoodCode'] != 9]

In [23]:
asa24 = asa24[asa24['FoodCode'] != 9]
asa24 = asa24[asa24['Food_Description'] != ' ']

In [24]:
asa24['FoodCode'].dropna(inplace=True)

In [25]:
old = new_codes['FoodCode'].tolist()

In [26]:
new = new_codes['NewFoodCode'].tolist()

In [27]:
keep = asa24[~asa24['FoodCode'].isin(old)]

In [28]:
match = asa24[asa24['FoodCode'].isin(old)]

In [29]:
match['FoodCode'].replace(old, new, inplace=True) #SettingwithcopyWarning

/Users/jules.larke/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [30]:
asa24_matched = pd.concat([match, keep], axis=0)

### Merge ASA24 updated foodcodes with FNDDS to ingredientize

In [32]:
#Load data
fndds = pd.read_csv('data/fndds/fndds_2018/fndds2018.csv')

#Rename column 'Food code' to 'FoodCode' for merging with ASA data
fndds.columns=['FoodCode',
               'fndds_description',
               'WWEIA Category number',
               'WWEIA Category description',
               'Seq num',
               'Ingredient code',
               'Ingredient description',
               'Ingredient weight (g)',
               'Retention code',
               'Moisture change (%)']

In [34]:
asa_fndds = pd.merge(asa24_matched, fndds, on='FoodCode')

In [35]:
asa_set = set(asa24['Food_Description'].unique())
asa_fndds_set = set(asa_fndds['Food_Description'].unique())

In [36]:
# 2 foods did not map to FNDDS2018
unmapped = asa_set.difference(asa_fndds_set)
unmapped

{'Oriental party mix, with peanuts, sesame sticks, chili rice crackers and fried green peas',
 'Vegetable sticks, breaded (including corn, carrots, and green beans)'}

In [38]:
#Rearrange columns of interest
cols_to_order = ['UserName',
 'FoodCode',
 'Food_Description',
 'fndds_description',
 'WWEIA Category number',
 'WWEIA Category description',
 'FoodAmt',                
 'Seq num',
 'Ingredient code',
 'Ingredient description',
 'Ingredient weight (g)']
new_columns = cols_to_order + (asa_fndds.columns.drop(cols_to_order).tolist())
asa_fndds = asa_fndds[new_columns]

In [23]:
asa_fndds.to_csv('asa24_ingredients/asa_fndds_matched_120721.csv', index=None)